# Derrida Coefficient

In this notebook we're going to figure out the code we need to calculate the Deridda coefficient before moving it into a script to run on all of ECA. I'm honestly not even sure the STGs give us an advantage here but they might make it easier. who knows.

I'm going to be basing my calculation off of $D_c$ as shown in Santosh's dissertation.

In [86]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from copy import copy, deepcopy

# will need this later
# converts an integer to a boolean representation as a list
def to_binary(n, digits=8):
    binary_digits = []
    for _ in range(digits):
        binary_digits.append(int(n % 2))
        n = int(n / 2)
    return binary_digits[::-1]

def to_decimal(b, digits):
    expos = np.arange(digits, 0, -1) - 1
    enc = 2**expos
    return np.array(b).T.dot(enc)

# which rule do we want to look at
rule = 54
cells = 6

STG = nx.read_graphml('../data/eca_stgs/' + str(rule) + '/' + str(cells) + '_cells.graphml')
to_decimal([1, 1, 1], 3)

7

In [87]:
# params for derrida drawn from Santosh's dissertation
# n/10 is not useful here so we'll try just 1?? maybe [1, 2]??
M = 1

def derrida(G, cells):
    """
    Parameters
    ----------
    G : nx.DiGraph
        state-transition graph for the system
    m : array-like (int) [not implemented]
        size of perturbations to consider
    
    Return
    ------
    D : float
        deridda coefficient
    """

    derrida = 0

    # we check this value over all posible states
    for state in range(2**cells):
        # get next state from graph
        next_state = next(G.neighbors(str(state)))
        # binary represenatation of initial state
        encoded_init = to_binary(state, cells)

        # for every possible perturbation
        for bit in range(cells):
            # flip a bit to make the perturbation
            perturb = deepcopy(encoded_init)
            if perturb[bit] == 0:
                perturb[bit] = 1
            elif perturb[bit] == 1:
                perturb[bit] = 0

            # get the next step in the perturbed state
            next_perturbed = next(G.neighbors(str(to_decimal(perturb, cells))))
            
            bin_next_state = np.array(to_binary(int(next_state), cells))
            bin_next_pert = np.array(to_binary(int(next_perturbed), cells))

            derrida += np.sum(~(bin_next_state == bin_next_pert))
    
    derrida /= 2* cells * 2**cells

    return derrida

derrida(STG, cells)

1.0

In [67]:
x = np.array(to_binary(9, 4))
y = np.array(to_binary(8, 4))
hamming = np.sum(~(x == y))
print(x)
print(y)
print(hamming)

[1 0 0 1]
[1 0 0 0]
1
